# droitGPT

# LLM: Qwen-1_8B-Chat-Int4

In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MAX_NEW_TOKENS = 8192
"""
Source: 
- context length: https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4
"""
MODEL_ID = "Qwen/Qwen-1_8B-Chat-Int4"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,device_map="auto",trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=MAX_NEW_TOKENS)
hf = HuggingFacePipeline(pipeline=pipe)

/home/adrian/Desktop/NLP/droitGPT/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.
Try importing flash-attention for faster inference...


# Model performance - English
Answer is good enough

In [2]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

 Electroencephalography (EEG) is a medical test used to measure electrical activity in the brain. It involves placing electrodes on different parts of the head and recording the electrical signals generated by neurons. EEG can be used to diagnose a wide range of neurological conditions, including epilepsy, migraines, and brain tumors.

The process of EEG is as follows:

1. The patient is placed in a safe position with their head resting on a table or pillow.
2. A series of electrodes are attached to the scalp, face, temples, and earlobes.
3. An EEG machine is set up and connected to an external power source.
4. The electrodes are turned on and the patient begins to breathe slowly.
5. The EEG machine records the electrical signals generated by neurons in the brain.
6. The data is then displayed on a screen for the physician to review.
7. The physician may use the data to diagnose specific neurological conditions or determine if a treatment plan is working effectively.

In summary, EEG i

# Model performance - French
Answer is good enough

In [3]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Réponse: Réfléchissons étape par étape."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "Qu’est-ce que l’électroencéphalographie ?"

print(chain.invoke({"question": question}))

 L'électroencephalographie (EEG) est une technique qui permet de mesurer les émotions, la pensée et les pensées artificielles (PA) dans un individu en utilisant des ressources physiques tels que les fosseaux du crâines, le ventromère du coeur et les réseaux du cerveau.

1. Émotion sensor(s): Les émotions sont souvent mesurées à travers des dispositifs électrocardiologiques ou des émotions à l'aide d'un écran intégral (EI). Ces dispositifs utilisent des réserves telles que des nodules pulmonaires, des artères et des réseaux musculaires pour montrer les signes émotionnels, comme la réflexion, la Feuille pleine et la forte bruit de l'气.
2. Sensor de puissance: Les sensors de puissance (SPS) sont des dispositifs conçus pour mesure la puissance des muscles lorsqu'ils sont utilisés pour défendre une position ou une action. Ils peuvent être équipés d'une télégraphie ou d'une oscilloscope pour visualiser les mouvements des muscles et les résultats obtenus par l'e胜ulation.
3. Réseaux du cerveau

# Test LLM with specific context
Good performance so far

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
Vous êtes un assistant spécialisé en codes juridiques français.

Utilisez les informations suivantes pour répondre à la question:
{context}

Question: {input}

Réponse: En utilisant le contexte, répondre à la question.
"""

prompt = PromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(hf, prompt)

In [5]:
from langchain_core.documents import Document

specific_context = """Le Peuple français proclame solennellement son attachement aux Droits de l'Homme et aux principes de la souveraineté nationale tels qu'ils ont été définis par la Déclaration de 1789, confirmée et complétée par le préambule de la Constitution de 1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de 2004.""" 

response = document_chain.invoke({
    "input": "Quels sont les éléments auxquels le Peuple français proclame solennellement son attachement dans ce texte ?",
    "context": [Document(page_content=specific_context)]
})
print(response)

Le Peuple français proclame solennellement son attachement aux Droits de l'Homme et aux principes de la souveraineté nationale tels qu'ils ont été définis par la Déclaration de 1789, confirmée et complétée par le préambule de la Constitution de 1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de 2004.



# Sentence Transformers
- Il faut trouver un modèle Sentence Transformers gratuit (donc pas de OpenAI Embeddings ni Ollama Embeddings)
- Ce modèle doit être pre-entrainé pour la comparaison semantique
- Ce modèle doit comprendre le français

Source https://www.sbert.net/docs/pretrained_models.html

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_id)

# Corpus
On va se limiter a 3 codes juridiques

## Clean Corpus

In [7]:
import os
import re

data_folder = "data"
relevant_files = ["travail.md", "education.md", "electoral.md"]

clean_data_folder_name = "clean_data"

if not os.path.exists(clean_data_folder_name):
        os.makedirs(clean_data_folder_name)

for file in os.listdir(data_folder):
    if file in relevant_files:
        file_path = data_folder + "/" + file 
        clean_file_path = clean_data_folder_name + "/clean_" + file
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

            # Transform textdata with re here
            pattern1 = re.compile(r'---.*?---', re.DOTALL) # remove title and date 
            text = re.sub(pattern1, '', text)
            
            pattern2 = re.compile(r'#+.*\n') # remove Markdown titles
            text = re.sub(pattern2, '', text)

            pattern2 = re.compile(r'\*\*.*\*\*') # remove Article titles
            text = re.sub(pattern2, '', text)

            pattern3 = re.compile(r'<div.*</div>') # remove html content (simple approach, usually tables)
            text = re.sub(pattern3, '', text)

            pattern4 = re.compile(r'([:;])\n+') # form paragraphs
            text = re.sub(pattern4,lambda x: x.group(1) + ' ', text)

            with open(clean_file_path, "w", encoding="utf-8") as f:
                f.write(text)

In [8]:
import os
import re

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document


CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0

data_folder = "clean_data"
documents = []
for file in os.listdir(data_folder):
    filename = os.fsdecode(data_folder + "/" + file)
    loader = TextLoader(filename)
    documents += loader.load()

docs = []
for doc in documents:
    texts = re.split('\n+', doc.page_content)
    texts = [text for text in texts if text]
    for text in texts:
        docs.append(Document(page_content = text, metadata=doc.metadata))

In [9]:
print(len(docs))
print()
print(docs[0])
print()
print(docs[100])

35102

page_content="Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et interprofessionnelle fait l'objet d'une concertation préalable avec les organisations syndicales de salariés et d'employeurs représentatives au niveau national et interprofessionnel en vue de l'ouverture éventuelle d'une telle négociation." metadata={'source': 'clean_data/clean_travail.md'}

page_content='Elles sont également applicables au personnel des personnes publiques employé dans les conditions du droit privé.' metadata={'source': 'clean_data/clean_travail.md'}


# Vector Database

In [10]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(docs, embeddings)

# Querying the Database
On voit que les docs récuperés de la base de données sont un peu similaires au requête (modèle Embeddings n'est pas très grand)

In [11]:
def print_context(docs_and_scores):
    for i, (doc, score) in enumerate(docs_and_scores):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [12]:
query = "L'éducation des jeunes sourds"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
L'utilisation de la langue des signes dans l'éducation des jeunes sourds est régie par les articles R. 351-21 à R. 351-26.

Top 2
Les établissements ou services relevant du 2° du I de l'article L. 312-1 du code de l'action sociale et des familles qui soit assurent en leur sein la scolarisation des jeunes sourds, soit contribuent à leur projet personnalisé de scolarisation lorsqu'ils sont scolarisés dans des écoles ou des établissements scolaires, ainsi que les établissements dont la création ou l'extension sont envisagées, élaborent un document annexé au projet d'établissement ou de service relatif aux conditions d'éducation et au parcours scolaire proposés aux jeunes sourds.

Top 3
Dans l'éducation et le parcours scolaire des jeunes sourds, la liberté de choix entre une communication bilingue, langue des signes et langue française, et une communication en langue française est de droit. Un décret en Conseil d'Etat fixe, d'une part, les conditions d'exercice de ce choix pour les j

In [13]:
query = "Contrat de travail à durée déterminée"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
III. – Bénéficie également de l'allocation de professionnalisation et de solidarité le travailleur involontairement privé d'emploi qui justifie de 507 heures de travail au cours des dix-huit mois qui précèdent la date de fin de la période d'indemnisation ouverte au titre de la clause de rattrapage prévue aux annexes mentionnées au I.

Top 2
Il peut être conclu afin de pourvoir un emploi permanent qui, par nature, comporte une alternance de périodes travaillées et de périodes non travaillées.

Top 3
La dérogation, prévue à l'article L. 5132-15-1 en cas de cumul avec un autre contrat de travail à temps partiel, à la durée hebdomadaire de travail minimale du titulaire d'un contrat à durée déterminée conclu en application de l'article L. 1242-3, est autorisée par le préfet, après examen par la structure d'insertion par l'activité économique qui emploie le salarié, de la situation de celui-ci au regard de l'emploi et des actions d'accompagnement et de formation conduites dans le cadre

In [14]:
query = "Le mandat de conseiller départemental est incompatible"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
Le mandat de conseiller départemental est incompatible, dans le département, avec les fonctions d'architecte départemental, d'ingénieur des travaux publics de l'Etat, chef de section principal ou chef de section des travaux publics de l'Etat chargé d'une circonscription territoriale de voirie, d'employé des bureaux de la préfecture ou d'une sous-préfecture et, généralement, de tous les agents salariés ou subventionnés sur les fonds départementaux.

Top 2
Pendant la durée de ses fonctions, le Contrôleur général des lieux de privation de liberté ne peut être candidat à un mandat de conseiller départemental s'il n'exerçait le même mandat antérieurement à sa nomination.

Top 3
Les conseillers départementaux déclarés démissionnaires d'office par application de l'article L. 3121-4 du code général des collectivités territoriales sont inéligibles pendant une année au conseil départemental.

Top 4
Le mandat de député est incompatible avec l'exercice de plus d'un des mandats énumérés ci-ap

# Use Vector Database as context depending on the input/question

In [15]:
def print_response(response):
    print("Answer:")
    print(response["answer"])
    print()
    print("Context:")
    contexts = response["context"]
    for i, doc in enumerate(contexts):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [16]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Test - Code Education

In [17]:
question = "Est-ce que l'education est la première priorité nationale en France?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

Answer:
Dans la lettre R. 451-2, prévoit l'article R. 368-1 des sections des article R. 368-1 des sections des articles R. 378-1 et les articles R. 378-2 des sections des articles R. 378-2 des articles R. 378-3, les décisions concernant la compétence académique de les élèves, incluant les décisions relatives aux décisions d'orientation, par le directeur général des établissements scolaires français à l'étranger, ont déclaré que l'education est la première priorité nationale en France.


Context:
Top 1
La scolarité accomplie par les élèves dans les établissements scolaires français à l'étranger est considérée, en vue de la poursuite de leurs études et de la délivrance des diplômes, comme effectuée en France dans un établissement d'enseignement public.

Top 2
La scolarité accomplie par les élèves dans les établissements d'enseignement placés à la suite des forces françaises et de l'élément civil stationnés en Allemagne est considérée, en vue de la poursuite de leurs études et de la déliv

# Test - Code Electoral

In [18]:
question = "Qu'est ce que tu connais sur le code electoral français?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

Answer:
Sont électeurs les français établis hors de Franceinscrits sur leslistes électorales consularies établies, révisées et contrôlées dans les conditions prévues au chapitre Ier du décret n° 2005-1613 du 22 décembre 2005 portant application de la loi organique n° 76-97 du 31 janvier 1976 relative aux listes électorales consulaires et au vote des français établs hors de France pour l'élection du Président de la République.
大切



Context:
Top 1
Sont électeurs les Français établis hors de France inscrits sur les listes électorales consulaires établies, révisées et contrôlées dans les conditions prévues au chapitre Ier du décret n° 2005-1613 du 22 décembre 2005 portant application de la loi organique n° 76-97 du 31 janvier 1976 relative aux listes électorales consulaires et au vote des Français établis hors de France pour l'élection du Président de la République.

Top 2
Tout Français et toute Française ayant la qualité d'électeur peut faire acte de candidature et être élu, sous réserve

# Test - Code Travail

In [23]:
question = "Est-ce qu'une organisation syndicale de salariés peut agir devant une juridiction civile?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

Answer:
Pour répondre à cette question, il faut utiliser les informations suivantes:

Les organisations syndicales représentatives dans l'entreprise utilis嫁给ter peuvent exercer en justice les actions civiles nées en vertu des dispositions du présent chapitre en faveur des salariés du groupement.

Lorsque vous utilisez ces informations pour répondre à la question, il semble qu'elles s'ent她们 fastamentes liées aux informations fournies pour permettre de réponses.
碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟碟$", NSMutable晚餐//////// fen荷兰 annoyed_parent轻轻地omasattach/pdf不断地belum할.timedeltaquisites地慢慢地reetings不断地François aux微软雅黑bt geral不断地 ave Lisbon巴黎份 replayירוש法国Inicioprés nive不慎_performance presente gamerelif cider qpux稿件 cow很好地łąchedulers